# SymPy - symbolické výpočty v Pythonu
**Na úvod**

Někteří z vás možná znáte nějaký systém pro *počítačovou algebru* (Computer Algebra Systems -- CAS), např. Maple, Mathematica, Derive, Maxima, Reduce. Pro Python existují dva velké projekty počítačové algebry:

* [SymPy](http://sympy.org) - modul který může být použit v jakémkoli Python programu a je dobře podporován v Jupyter Notebook. 
* [Sage](http://www.sagemath.org/) - toto je už kompletní (a velice obsáhlý) systém, který si klade za cíl být open source konkurentem komerčním produktům.

My se tady podíváme na některé základní možnosti SymPy.

In [ ]:
import sympy

In [ ]:
sympy.init_printing()

## Definujeme symboly

Pro symbolické výpočty potřebujeme pochopitelně symboly, tak jak jsme zvyklí už z matematiky na základní škole. V Pythonu samotném máme sice proměnné, které jsou v podstatě také symboly, ale operace s nimy se řídí zcela jinými pravidly než potřebujeme pro symbolické výpočty. Naštěstí tu je třída `sympy.Symbol`.

In [ ]:
x = sympy.Symbol('x')
x

Co když napíšeme něco trochu složitějšího.

In [ ]:
expression = (sympy.pi + x) / 2
expression

In [ ]:
# co jsme to vůbec dostali za typ
type(expression)


Můžeme také přičadit symbolům nějaké vlastnosti (to se pak pochopitelně může projevit v dalších výpočtech).

In [ ]:
a = sympy.Symbol('a', real=True)
a.is_real

In [ ]:
b = sympy.Symbol('b', positive=True)

In [ ]:
b.is_negative

In [ ]:
b > 0

### Zlomky

In [ ]:
r1 = sympy.Rational(4,5)
r2 = sympy.Rational(5,4)
r1, r2

In [ ]:
r1 + r2

## Vyčíslování

In [ ]:
y = (x + sympy.pi)**2
y

Numerickou hodnotu můžeme získat pomocí funkce `N`. Často také využijeme metodu `subs`:

In [ ]:
expression = y.subs(x, 2)
print(expression)

sympy.N(expression, n=5)


To samé pomocí metody `evalf`. Pro obojí můžeme zadat počet platných číslic.

In [ ]:
sympy.pi.evalf(100)

Pokud chceme vytvořit ze symbolického výrazu funkci, použijeme `lambdify`:

In [ ]:
# první argument je seznam proměnných (podobně jako pro lambda funkce)
f_sympy = (x + sympy.pi)**2
print(f_sympy)

f_x = sympy.lambdify([x], f_sympy)
print(f_x)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

xa = np.linspace(-10, 10)
plt.plot(xa, f_x(xa))

## Symbolické úpravy

Toto je velice důležitá aplikace, která nám může v mnoha případech ušetřit nemálo práce.

### Expand a factor

Začněme pracovat s polynomem, zadaným jako

In [ ]:
y = (x+1)*(x+2)*(x+3)
y

Polynom rozvineme pomocí `expand`:

In [ ]:
z = sympy.expand(y)
z

Pomocí `factor` můžeme dostat zpět původní faktorizovaný výraz.

In [ ]:
sympy.factor(z)

`expand` můžeme použít i pro trigonometrické funkce:

In [ ]:
sympy.expand(sympy.sin(a + b), trig=True)

### Zjednodušování pomocí `simplify`

In [ ]:
# tohle by měla být hračka
expression = sympy.sin(a)**2 + sympy.cos(a)**2
expression


In [ ]:
sympy.simplify(expression)


## Derivace a integrály

SymPy umí symbolicky derivovat (je tedy aspoň tak dobrý jako cvičená opice) a i integrovat.

In [ ]:
y = (x**2 + sympy.sin(x))**2
y

In [ ]:
sympy.diff(y, x)

Derivovat můžeme i funkce více proměnných.

In [ ]:
x = sympy.Symbol('x')
y = sympy.Symbol('y')
z = sympy.cos(y) * (x**3 + 2*x**2*y)
z

Tohle spočítá

$\displaystyle \frac{{{{\rm{d}}^3}z}}{{{\rm{d}}x{\rm{d}}{y^2}}} $

In [ ]:
sympy.diff(z, x, 1, y, 2)

### Integrace

In [ ]:
f = sympy.sin(x * y) * sympy.cos(x)
f

In [ ]:
sympy.integrate(f, x)

In [ ]:
sympy.integrate(f, y)

## Řešení rovnic
Rovnice můžeme řešit pomocí `solve`, prvním argumentem je list výrazů, které musí být rovny nule, druhým je touple symbolů, pro které chceme nalézt řešení.

**Příklad:** Hledání předpisu lineární funkce, která ma na daném trojúhelníku hodnotu 0 ve dvou bodech a 1 ve třetím bodě.

In [ ]:
import sympy as sp

# proměnné v souřadnicovém systému
x, y = sp.symbols('x y')

# proměnné reprezentující souřadnice bodů
x1, y1, x2, y2, x3, y3 = sp.symbols('x1 y1 x2 y2 x3 y3')

# proměnné reprezentující parametry lineární funkce ve 2D
a, b, c = sp.symbols('a b c')

# takto vypadá obecně lineární funkce ve 2D
phi = a * x + b * y + c

# systém rovnic pro 3 body, v prvním bodě je phi = 1 v druhém bodě je phi = 0 a v třetím bodě je phi = 0
conditions_phi = [
    phi.subs({x: x1, y: y1}) - 1,
    phi.subs({x: x2, y: y2}),
    phi.subs({x: x3, y: y3})
]
conditions_phi


In [ ]:
# spočteme, jaké musí být hodnoty a, b a c, aby byl systém rovnic splněn
coeffs_phi = sp.solve(conditions_phi, (a, b, c))

coeffs_phi


In [ ]:
# použijeme řešení pro vytvoření hledané funkce
phi = phi.subs(coeffs_phi)
phi


## Generace kódu

Automatická generace kódu je funkcionalita, kterou oceníme ve chvíli, kdy cheme implementovat naše analytické výsledky v numerické simulaci. Místo abychom začali ručně přepisovat do programovacího jazyka jako je např. Fortran nebo C, může SymPy tuto nezábavnou práci udělat za nás. Navíc při tom neudělá chyby.

In [ ]:
# řekněme že chceme někde použít tento výsledek
f = sympy.sin( x * y**2) * sympy.exp(y)
f

In [ ]:
import sympy.utilities.codegen

In [ ]:
# exportujeme do jazyka Fortran
f_source = sympy.utilities.codegen.codegen(("f_fortran", f), "F95", "f_fortran")
print(f_source[0][1])

In [ ]:
# exportujeme do jazyka C
f_source = sympy.utilities.codegen.codegen(("f_C", f), "C", "f_C")
print(f_source[0][1])

In [ ]:
# exportujeme do jazyka Octave/Matlab
f_source = sympy.utilities.codegen.codegen(("f_octave", f), "Octave", "f_octave")
print(f_source[0][1])

In [ ]:
help(sympy.utilities.codegen.codegen)


## Další možnosti SymPy

Ukázali jsme si základy práce se symbolickými výpočty pomocí SymPy. Není v našich silách ukázat, co všechno SymPy umí -- je toho opravdu hodně. 

Přehled můžeme získat např. pomocí následujících odkazů:

* [SymPy Tutorial](http://docs.sympy.org/latest/tutorial/index.html) - základní tutoriál
* [SymPy Documentation](http://docs.sympy.org/latest/index.html) - kompletní dokumentace
* [SymPy Live](http://live.sympy.org/) - online SymPy notebook
* [SymPy GitHub](https://github.com/sympy/sympy/wiki) - SymPy na GitHubu

### Seznam témat které SymPy pokrývá

* SymPy Core
* Combinatorics Module
* Number Theory
* Concrete Mathematics
* Numerical evaluation
* Functions Module
* Geometry Module
* Geometric Algebra Module
* Geometric Algebra Module for SymPy
* Extended LaTeXModule for SymPy
* Symbolic Integrals
* Numeric Integrals
* Logic Module
* Matrices
* Mpmath
* Polynomials Manipulation Module
* Printing System
* Plotting Module
* Pyglet Plotting Module
* Assumptions module
* Term rewriting
* Series Expansions
* Sets
* Simplify
* Details on the Hypergeometric Function Expansion Module
* Statistics
* Stats
* ODE
* PDE
* Solvers
* Tensor Module
* Utilities
* Parsing input
* Physics Module
* Category Theory Module
* Differential Geometry Module
* Contributions to docs